In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers biopython kaggle
print("✅ Đã cài đặt xong thư viện!")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 53.9 MB/s eta 0:00:00
✅ Đã cài đặt xong thư viện!


In [16]:
import os
from google.colab import files

# Upload file kaggle.json
print("Vui lòng upload file kaggle.json của bạn:")
files.upload()

# Cấu hình Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Tải dữ liệu cuộc thi (Sẽ mất khoảng 1-2 phút)
# print("⏳ Đang tải dữ liệu CAFA 6...")
# !kaggle competitions download -c cafa-6-protein-function-prediction
# !unzip -q cafa-6-protein-function-prediction.zip -d /content/cafa6_data
# print("✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data")

Vui lòng upload file kaggle.json của bạn:


Saving kaggle.json to kaggle.json


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import gc
import os
import sys
from tqdm.auto import tqdm

# ============================================================================
# CONFIGURATION
# ============================================================================
CONFIG = {
    'EMBEDDINGS': {
        'ankh': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_test_FINAL_embeddings.npy',
        },
        'esm': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh/esm2_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh/esm2_test_FINAL_embeddings.npy',
        },
        'prot_t5': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/Prot_T5_Aligned/protT5_xl_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/Prot_T5_Aligned/protT5_xl_test_FINAL_embeddings.npy',
        },
        # 'protBERT': {
        #     'train': '/content/drive/MyDrive/CAFA6_Embeddings/protBERT_Aligned/protBERT_train_FINAL_embeddings.npy',
        #     'test':  '/content/drive/MyDrive/CAFA6_Embeddings/protBERT_Aligned/protBERT_test_FINAL_embeddings.npy',
        # },
    },

    'TRAIN_ID_PATH': '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_train_FINAL_ids.npy',
    'TEST_ID_PATH':  '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_test_FINAL_ids.npy',

    'SAVE_DIR': '/content/drive/MyDrive/CAFA6_Results/prott5_esm2_ankh_Run',
    'TRAIN_TERMS': '/content/cafa6_data/Train/train_terms.tsv',
    'IA_FILE': '/content/cafa6_data/IA.tsv',
    'TEST_FASTA': '/content/cafa6_data/Test/testsuperset.fasta',
    'OBO_FILE': '/content/cafa6_data/Train/go-basic.obo',

    # Model Params
    'TOP_K_LABELS': 10000, # Giữ mức này để model học tốt
    # 'MIN_FREQ': 3,

    # Input sẽ là 768 (Ankh) + 1280 (ESM) = 2048
    'ENCODER_LAYERS': [2048, 1024, 512],

    'DROPOUT_RATE': 0.4, # Tăng dropout vì input lớn
    'EPOCHS': 50,
    'BATCH_SIZE': 128,
    'LEARNING_RATE': 2e-4, # Giảm LR một chút cho ổn định
    'LABEL_SMOOTHING': 0.1,

    # Inference Params (Chuẩn 0.27 điểm)
    'MIN_CONFIDENCE': 0.01,
    'MAX_PREDS_PER_PROTEIN': 20,

    'SEED': 42,
    'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu'
}

os.makedirs(CONFIG['SAVE_DIR'], exist_ok=True)
print(f"🚀 CAFA 6 - DUAL MODEL (ANKH + ESM) | Device: {CONFIG['DEVICE']}")

torch.manual_seed(CONFIG['SEED'])
np.random.seed(CONFIG['SEED'])

def load_go_aspect_mapping(obo_path):
    print(f"📖 Parsing OBO file: {obo_path}...")
    namespace_map = {
        'biological_process': 'BPO',
        'molecular_function': 'MFO',
        'cellular_component': 'CCO'
    }
    mapping = {}
    current_id = ""
    with open(obo_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('id: GO:'):
                current_id = line.split('id: ')[1]
            elif line.startswith('namespace:'):
                if current_id:
                    full_ns = line.split('namespace: ')[1]
                    mapping[current_id] = namespace_map.get(full_ns, 'UNKNOWN')
    print(f"✅ Loaded aspect mapping for {len(mapping):,} terms.")
    return mapping

# ============================================================================
# 1. MEMORY-SAFE DATASET (CHÌA KHÓA ĐỂ KHÔNG TRÀN RAM)
# ============================================================================
class MultiSourceDataset(Dataset):
    def __init__(self, embedding_paths_dict, y_tensor=None, indices=None):
        self.mmaps = {}
        self.keys = list(embedding_paths_dict.keys())

        # Load mmap
        for name, path in embedding_paths_dict.items():
            self.mmaps[name] = np.load(path, mmap_mode='r')

        # Base length
        first_key = self.keys[0]
        self.total_len = len(self.mmaps[first_key])

        self.indices = indices if indices is not None else np.arange(self.total_len)
        self.y = y_tensor

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        real_idx = self.indices[idx]

        inputs = []
        # Load từng vector -> List of Tensors
        for key in self.keys:
            vec = torch.from_numpy(self.mmaps[key][real_idx].copy()).float()
            inputs.append(vec)

        if self.y is not None:
            return inputs, self.y[real_idx]
        return (inputs,)

# ============================================================================
# 2. LOAD DATA & PROCESS LABELS
# ============================================================================
print("\n[1/5] Checking Files...")
train_ids = np.load(CONFIG['TRAIN_ID_PATH'])
print(f"   ✓ Train IDs: {len(train_ids)}")

print("\n[2/5] Processing Labels (IA Strategy)...")
# 1. Load Terms & IA
df_terms = pd.read_csv(CONFIG['TRAIN_TERMS'], sep='\t', header=0, names=['EntryID', 'term', 'aspect'])
df_ia = pd.read_csv(CONFIG['IA_FILE'], sep='\t', names=['term', 'ia'])
ia_dict = dict(zip(df_ia['term'], df_ia['ia']))

# 2. Tính Score
term_counts = df_terms['term'].value_counts().reset_index()
term_counts.columns = ['term', 'freq']

# print(f"   Original terms count: {len(term_counts)}")
# term_counts = term_counts[term_counts['freq'] >= CONFIG['MIN_FREQ']]
# print(f"   -> After filtering (Freq >= {CONFIG['MIN_FREQ']}): {len(term_counts)} terms")

term_counts['ia'] = term_counts['term'].map(ia_dict).fillna(0.0)
term_counts['score'] = term_counts['freq'] * term_counts['ia']

# 3. Chọn Top K
top_terms_df = term_counts.sort_values(by='score', ascending=False).head(CONFIG['TOP_K_LABELS'])
top_terms = top_terms_df['term'].tolist()
print(f"   ✓ Selected {len(top_terms)} terms.")

obo_path = CONFIG['OBO_FILE']
go_aspect_map = load_go_aspect_mapping(obo_path)

aspect_groups = {
    'MFO': [],
    'BPO': [],
    'CCO': []
}

for term in top_terms:
    aspect = go_aspect_map.get(term, 'UNKNOWN')
    if aspect in aspect_groups:
        aspect_groups[aspect].append(term)

print(f"📊 Aspect Distribution:")
print(f"   - MFO (Molecular Function): {len(aspect_groups['MFO']):,} labels")
print(f"   - CCO (Cellular Component): {len(aspect_groups['CCO']):,} labels")
print(f"   - BPO (Biological Process): {len(aspect_groups['BPO']):,} labels")

# 4. Map ID -> Terms
df_filtered = df_terms[df_terms['term'].isin(top_terms)]
id_to_terms = df_filtered.groupby('EntryID')['term'].apply(list).to_dict()

# Dọn dẹp
del df_terms, df_filtered, df_ia, term_counts, top_terms_df
gc.collect()

# ============================================================================
# 5. MODEL & TRAINING
# ============================================================================
print("\n[5/5] Building Model...")

class EncoderBlock(nn.Module):
    def __init__(self, in_dim, layers_config, dropout):
        super().__init__()
        layers = []

        # 1. LAYER NORM ĐẦU VÀO (BẮT BUỘC)
        # Để cân bằng 'âm lượng' giữa ESM (hét to) và Ankh (nói nhỏ)
        layers.append(nn.LayerNorm(in_dim))

        prev_dim = in_dim
        for i, dim in enumerate(layers_config):
            layers.extend([
                nn.Linear(prev_dim, dim),
                nn.BatchNorm1d(dim), # Ổn định training
                nn.GELU(),           # Hiện đại hơn ReLU
                nn.Dropout(dropout)
            ])
            prev_dim = dim

        self.net = nn.Sequential(*layers)
        self.out_dim = prev_dim

    def forward(self, x):
        return self.net(x)

# Module lọc nhiễu (Attention)
class SEBlock(nn.Module):
    """Squeeze-and-Excitation Block để lọc nhiễu sau khi gộp"""
    def __init__(self, in_dim, reduction=4):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_dim, in_dim // reduction, bias=False),
            nn.ReLU(),
            nn.Linear(in_dim // reduction, in_dim, bias=False),
            nn.Sigmoid() # Tạo ra mask từ 0 đến 1
        )

    def forward(self, x):
        # x shape: [Batch, Dim]
        # Attention weight: [Batch, Dim]
        w = self.fc(x)
        # Nhân trọng số vào x: Cái nào quan trọng thì giữ, rác thì nhân với 0
        return x * w

class MultiModalNet(nn.Module):
    def __init__(self, input_dims_list, encoder_layers, dropout, num_classes):
        super().__init__()

        self.encoders = nn.ModuleList()
        self.fusion_input_dim = 0

        print("\n🏗️ Building Advanced Architecture:")

        # 1. Xây dựng các nhánh Encoder
        for i, in_dim in enumerate(input_dims_list):
            print(f"   ➤ Branch {i+1}: Input {in_dim} -> Output {encoder_layers[-1]}")
            enc = EncoderBlock(in_dim, encoder_layers, dropout)
            self.encoders.append(enc)
            self.fusion_input_dim += enc.out_dim

        print(f"   ➤ Fusion Dim: {self.fusion_input_dim}")

        # 2. SE-Block (Bộ lọc thông minh)
        self.attention_filter = SEBlock(self.fusion_input_dim)

        # 3. Layer tổng hợp cuối cùng
        self.head = nn.Sequential(
            nn.BatchNorm1d(self.fusion_input_dim),
            nn.Linear(self.fusion_input_dim, 512),
            nn.GELU(),
            nn.Dropout(dropout),

            # Thêm một lớp nữa để tăng khả năng học
            nn.Linear(512, 512),
            nn.GELU(),
            nn.Dropout(dropout),

            nn.Linear(512, num_classes)
        )

    def forward(self, inputs_list):
        features = []
        # Đi qua từng nhánh
        for i, encoder in enumerate(self.encoders):
            feat = encoder(inputs_list[i])
            features.append(feat)

        # Gộp lại (Concatenate)
        combined = torch.cat(features, dim=1)

        # LỌC NHIỄU (Điểm khác biệt lớn nhất)
        # Mạng sẽ tự học cách "tắt tiếng" các đặc trưng rác từ ProtT5 nếu nó thấy không cần thiết
        refined = self.attention_filter(combined)

        return self.head(refined)

train_paths = {k: v['train'] for k, v in CONFIG['EMBEDDINGS'].items()}
aspect_names = ['MFO', 'CCO', 'BPO']
temp_dataset = MultiSourceDataset(train_paths, indices=[0])
sample_inputs = temp_dataset[0][0] # Get inputs of first sample
INPUT_DIMS_LIST = [x.shape[0] for x in sample_inputs]
print(f"   ✓ Detected Input Dims: {INPUT_DIMS_LIST}")
del temp_dataset

# Lưu thông tin để dùng cho Inference
trained_models_meta = []

for aspect in aspect_names:
    terms = aspect_groups[aspect]
    if len(terms) == 0: continue

    print(f"\n" + "="*60)
    print(f"🧬 TRAINING ASPECT: {aspect} ({len(terms)} labels)")
    print("="*60)

    # --- A. PREPARE Y TENSOR CHO ASPECT NÀY ---
    print(f"   Shape mapping for {aspect}...")
    mlb = MultiLabelBinarizer(classes=terms, sparse_output=True)
    mlb.fit([terms])

    # Chỉ transform các nhãn thuộc Aspect này
    # Lưu ý: id_to_terms chứa tất cả, mlb sẽ tự ignore các nhãn không thuộc 'terms'
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        y_labels_list = [id_to_terms.get(pid, []) for pid in train_ids]
        y_sparse = mlb.transform(y_labels_list)

    # Convert to Tensor
    y_binary = y_sparse.astype(np.float32).toarray()
    y_tensor = torch.from_numpy(y_binary)

    # Label Smoothing
    if CONFIG['LABEL_SMOOTHING'] > 0:
        y_tensor.mul_(1 - CONFIG['LABEL_SMOOTHING']).add_(CONFIG['LABEL_SMOOTHING'] / len(terms))

    # Pos Weight (Optional - Re-calculate per aspect)
    weights_list = [ia_dict.get(t, 0.0) for t in terms]
    pos_weight = torch.tensor(weights_list, dtype=torch.float32).to(CONFIG['DEVICE'])

    # --- B. DATALOADERS ---
    indices = np.arange(len(y_tensor))
    train_idx, val_idx = train_test_split(indices, test_size=0.15, random_state=CONFIG['SEED'])

    train_dataset = MultiSourceDataset(train_paths, y_tensor, indices=train_idx)
    val_dataset   = MultiSourceDataset(train_paths, y_tensor, indices=val_idx)

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=True, num_workers=2)
    val_loader   = DataLoader(val_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=False, num_workers=2)

    # Auto detect input dims (Chỉ làm lần đầu hoặc làm lại cũng được)
    # sample_inputs, _ = train_dataset[0]
    # INPUT_DIMS_LIST = [x.shape[0] for x in sample_inputs]

    # --- C. BUILD MODEL ---
    # Reset model mới cho mỗi Aspect
    model = MultiModalNet(
        input_dims_list = INPUT_DIMS_LIST,
        encoder_layers  = CONFIG['ENCODER_LAYERS'],
        dropout         = CONFIG['DROPOUT_RATE'],
        num_classes     = len(terms) # Output dim thay đổi theo Aspect
    ).to(CONFIG['DEVICE'])

    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['LEARNING_RATE'], weight_decay=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

    # --- D. TRAIN ---
    best_val_loss = float('inf')
    model_save_path = f"{CONFIG['SAVE_DIR']}/model_{aspect}.pth"

    for epoch in range(CONFIG['EPOCHS']):
        model.train()
        train_loss = 0
        for X_b, y_b in train_loader:
            X_b, y_b = [x.to(CONFIG['DEVICE']) for x in X_b], y_b.to(CONFIG['DEVICE'])
            optimizer.zero_grad()
            logits = model(X_b)
            loss = criterion(logits, y_b)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()

        avg_train = train_loss / len(train_loader)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_b, y_b in val_loader:
                X_b, y_b = [x.to(CONFIG['DEVICE']) for x in X_b], y_b.to(CONFIG['DEVICE'])
                logits = model(X_b)
                loss = criterion(logits, y_b)
                val_loss += loss.item()

        avg_val = val_loss / len(val_loader)
        scheduler.step(avg_val)

        print(f"   Epoch {epoch+1:02d} | Train: {avg_train:.4f} | Val: {avg_val:.4f}")

        if avg_val < best_val_loss:
            best_val_loss = avg_val
            torch.save(model.state_dict(), model_save_path)

    print(f"   🏆 Saved best {aspect} model to {model_save_path}")

    # Lưu meta data
    trained_models_meta.append({
        'aspect': aspect,
        'path': model_save_path,
        'terms': terms,
        'num_classes': len(terms)
    })

    # Dọn dẹp RAM
    del model, train_loader, val_loader, train_dataset, val_dataset, y_tensor, y_sparse
    gc.collect()
    torch.cuda.empty_cache()

# ============================================================================
# 4. INFERENCE (MULTI-MODEL MERGE)
# ============================================================================
print("\n🔮 PREDICTING WITH MULTIPLE MODELS...")

test_paths = {k: v['test'] for k, v in CONFIG['EMBEDDINGS'].items()}
test_ids = np.load(CONFIG['TEST_ID_PATH'])

# Dataset Test (Dùng chung cho cả 3 model)
test_dataset = MultiSourceDataset(test_paths)
test_loader = DataLoader(test_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=False, num_workers=2)

# Load 3 Model vào list (Deep Learning model nhẹ, load cả 3 vào VRAM ok)
# Nếu VRAM quá yếu (T4 thường chịu được), thì có thể load-predict-unload từng cái
loaded_models = []
all_combined_terms = []

print("   Loading models...")
for meta in trained_models_meta:
    # Build lại kiến trúc
    m = MultiModalNet(
        input_dims_list = INPUT_DIMS_LIST,
        encoder_layers  = CONFIG['ENCODER_LAYERS'],
        dropout         = CONFIG['DROPOUT_RATE'],
        num_classes     = meta['num_classes']
    ).to(CONFIG['DEVICE'])

    m.load_state_dict(torch.load(meta['path']))
    m.eval()
    loaded_models.append(m)
    all_combined_terms.extend(meta['terms']) # Ghép tên nhãn nối tiếp nhau

print(f"   Total combined labels: {len(all_combined_terms)}")

submission_path = f"{CONFIG['SAVE_DIR']}/submission.tsv"
n_predictions = 0
current_idx = 0

with open(submission_path, 'w') as f:
    with torch.no_grad():
        for (X_b,) in tqdm(test_loader, desc="Inference"):
            X_b = [x.to(CONFIG['DEVICE']) for x in X_b]

            # 1. Predict từng Model
            batch_preds_list = []
            for model in loaded_models:
                logits = model(X_b)
                probs = torch.sigmoid(logits).cpu().numpy() # [Batch, Num_Classes_Aspect]
                batch_preds_list.append(probs)

            # 2. Ghép ngang kết quả (Concatenate)
            # [Batch, MFO] + [Batch, CCO] + [Batch, BPO] -> [Batch, Total]
            batch_full_probs = np.hstack(batch_preds_list)

            # 3. Ghi file (Logic cũ)
            ids_batch = test_ids[current_idx : current_idx + len(batch_full_probs)]
            current_idx += len(batch_full_probs)

            for i, pid in enumerate(ids_batch):
                probs = batch_full_probs[i]

                # TOP-K
                top_k = CONFIG['MAX_PREDS_PER_PROTEIN']
                if len(probs) > top_k:
                    ind = np.argpartition(probs, -top_k)[-top_k:]
                else:
                    ind = np.arange(len(probs))

                ind = ind[np.argsort(probs[ind])][::-1]

                for idx in ind:
                    score = probs[idx]
                    if score > CONFIG['MIN_CONFIDENCE']:
                        # Map đúng index tổng
                        term = all_combined_terms[idx]
                        f.write(f"{pid}\t{term}\t{score:.3f}\n")
                        n_predictions += 1

            del batch_full_probs, batch_preds_list, X_b

print(f"\n✅ DONE! File: {submission_path}")
print(f"✅ Total Predictions: {n_predictions:,}")

🚀 CAFA 6 - DUAL MODEL (ANKH + ESM) | Device: cuda

[1/5] Checking Files...
   ✓ Train IDs: 82404

[2/5] Processing Labels (IA Strategy)...
   ✓ Selected 10000 terms.
📖 Parsing OBO file: /content/cafa6_data/Train/go-basic.obo...
✅ Loaded aspect mapping for 48,101 terms.
📊 Aspect Distribution:
   - MFO (Molecular Function): 3,465 labels
   - CCO (Cellular Component): 1,612 labels
   - BPO (Biological Process): 4,923 labels

[5/5] Building Model...
   ✓ Detected Input Dims: [768, 1280, 1024]

🧬 TRAINING ASPECT: MFO (3465 labels)
   Shape mapping for MFO...

🏗️ Building Advanced Architecture:
   ➤ Branch 1: Input 768 -> Output 512
   ➤ Branch 2: Input 1280 -> Output 512
   ➤ Branch 3: Input 1024 -> Output 512
   ➤ Fusion Dim: 1536
   Epoch 01 | Train: 0.0355 | Val: 0.0057
   Epoch 02 | Train: 0.0061 | Val: 0.0056
   Epoch 03 | Train: 0.0059 | Val: 0.0054
   Epoch 04 | Train: 0.0057 | Val: 0.0053
   Epoch 05 | Train: 0.0055 | Val: 0.0051
   Epoch 06 | Train: 0.0054 | Val: 0.0050
   Epoch 07

Inference:   0%|          | 0/1753 [00:00<?, ?it/s]


✅ DONE! File: /content/drive/MyDrive/CAFA6_Results/prott5_esm2_ankh_Run/submission.tsv
✅ Total Predictions: 4,308,204


In [13]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import gc
import os
import sys
from tqdm.auto import tqdm

# ============================================================================
# CONFIGURATION
# ============================================================================
CONFIG = {
    'EMBEDDINGS': {
        'ankh': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_test_FINAL_embeddings.npy',
        },
        'esm': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh/esm2_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh/esm2_test_FINAL_embeddings.npy',
        },
        'prot_t5': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/Prot_T5_Aligned/protT5_xl_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/Prot_T5_Aligned/protT5_xl_test_FINAL_embeddings.npy',
        },
        # 'protBERT': {
        #     'train': '/content/drive/MyDrive/CAFA6_Embeddings/protBERT_Aligned/protBERT_train_FINAL_embeddings.npy',
        #     'test':  '/content/drive/MyDrive/CAFA6_Embeddings/protBERT_Aligned/protBERT_test_FINAL_embeddings.npy',
        # },
    },

    'TRAIN_ID_PATH': '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_train_FINAL_ids.npy',
    'TEST_ID_PATH':  '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_test_FINAL_ids.npy',

    'SAVE_DIR': '/content/drive/MyDrive/CAFA6_Results/prott5_esm2_ankh_Run',
    'TRAIN_TERMS': '/content/cafa6_data/Train/train_terms.tsv',
    'IA_FILE': '/content/cafa6_data/IA.tsv',
    'TEST_FASTA': '/content/cafa6_data/Test/testsuperset.fasta',
    'OBO_FILE': '/content/cafa6_data/Train/go-basic.obo',

    # Model Params
    'TOP_K_LABELS': 10000, # Giữ mức này để model học tốt
    # 'MIN_FREQ': 3,

    # Input sẽ là 768 (Ankh) + 1280 (ESM) = 2048
    'ENCODER_LAYERS': [2048, 1024, 512],

    'DROPOUT_RATE': 0.4, # Tăng dropout vì input lớn
    'EPOCHS': 50,
    'BATCH_SIZE': 128,
    'LEARNING_RATE': 2e-4, # Giảm LR một chút cho ổn định
    'LABEL_SMOOTHING': 0.1,

    # Inference Params (Chuẩn 0.27 điểm)
    'MIN_CONFIDENCE': 0.01,
    'MAX_PREDS_PER_PROTEIN': 20,

    'SEED': 42,
    'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu'
}

os.makedirs(CONFIG['SAVE_DIR'], exist_ok=True)
print(f"🚀 CAFA 6 - DUAL MODEL (ANKH + ESM) | Device: {CONFIG['DEVICE']}")

torch.manual_seed(CONFIG['SEED'])
np.random.seed(CONFIG['SEED'])

def load_go_aspect_mapping(obo_path):
    print(f"📖 Parsing OBO file: {obo_path}...")
    namespace_map = {
        'biological_process': 'BPO',
        'molecular_function': 'MFO',
        'cellular_component': 'CCO'
    }
    mapping = {}
    current_id = ""
    with open(obo_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('id: GO:'):
                current_id = line.split('id: ')[1]
            elif line.startswith('namespace:'):
                if current_id:
                    full_ns = line.split('namespace: ')[1]
                    mapping[current_id] = namespace_map.get(full_ns, 'UNKNOWN')
    print(f"✅ Loaded aspect mapping for {len(mapping):,} terms.")
    return mapping

# ============================================================================
# 1. MEMORY-SAFE DATASET (CHÌA KHÓA ĐỂ KHÔNG TRÀN RAM)
# ============================================================================
class MultiSourceDataset(Dataset):
    def __init__(self, embedding_paths_dict, y_tensor=None, indices=None):
        self.mmaps = {}
        self.keys = list(embedding_paths_dict.keys())

        # Load mmap
        for name, path in embedding_paths_dict.items():
            self.mmaps[name] = np.load(path, mmap_mode='r')

        # Base length
        first_key = self.keys[0]
        self.total_len = len(self.mmaps[first_key])

        self.indices = indices if indices is not None else np.arange(self.total_len)
        self.y = y_tensor

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        real_idx = self.indices[idx]

        inputs = []
        # Load từng vector -> List of Tensors
        for key in self.keys:
            vec = torch.from_numpy(self.mmaps[key][real_idx].copy()).float()
            inputs.append(vec)

        if self.y is not None:
            return inputs, self.y[real_idx]
        return (inputs,)

# ============================================================================
# 2. LOAD DATA & PROCESS LABELS
# ============================================================================
print("\n[1/5] Checking Files...")
train_ids = np.load(CONFIG['TRAIN_ID_PATH'])
print(f"   ✓ Train IDs: {len(train_ids)}")

print("\n[2/5] Processing Labels (IA Strategy)...")
# 1. Load Terms & IA
df_terms = pd.read_csv(CONFIG['TRAIN_TERMS'], sep='\t', header=0, names=['EntryID', 'term', 'aspect'])
df_ia = pd.read_csv(CONFIG['IA_FILE'], sep='\t', names=['term', 'ia'])
ia_dict = dict(zip(df_ia['term'], df_ia['ia']))

# 2. Tính Score
term_counts = df_terms['term'].value_counts().reset_index()
term_counts.columns = ['term', 'freq']

# print(f"   Original terms count: {len(term_counts)}")
# term_counts = term_counts[term_counts['freq'] >= CONFIG['MIN_FREQ']]
# print(f"   -> After filtering (Freq >= {CONFIG['MIN_FREQ']}): {len(term_counts)} terms")

term_counts['ia'] = term_counts['term'].map(ia_dict).fillna(0.0)
term_counts['score'] = term_counts['freq'] * term_counts['ia']

# 3. Chọn Top K
top_terms_df = term_counts.sort_values(by='score', ascending=False).head(CONFIG['TOP_K_LABELS'])
top_terms = top_terms_df['term'].tolist()
print(f"   ✓ Selected {len(top_terms)} terms.")

obo_path = CONFIG['OBO_FILE']
go_aspect_map = load_go_aspect_mapping(obo_path)

aspect_groups = {
    'MFO': [],
    'BPO': [],
    'CCO': []
}

for term in top_terms:
    aspect = go_aspect_map.get(term, 'UNKNOWN')
    if aspect in aspect_groups:
        aspect_groups[aspect].append(term)

print(f"📊 Aspect Distribution:")
print(f"   - MFO (Molecular Function): {len(aspect_groups['MFO']):,} labels")
print(f"   - CCO (Cellular Component): {len(aspect_groups['CCO']):,} labels")
print(f"   - BPO (Biological Process): {len(aspect_groups['BPO']):,} labels")

# 4. Map ID -> Terms
df_filtered = df_terms[df_terms['term'].isin(top_terms)]
id_to_terms = df_filtered.groupby('EntryID')['term'].apply(list).to_dict()

# Dọn dẹp
del df_terms, df_filtered, df_ia, term_counts, top_terms_df
gc.collect()

# ============================================================================
# 5. MODEL & TRAINING
# ============================================================================
print("\n[5/5] Building Model...")

class EncoderBlock(nn.Module):
    def __init__(self, in_dim, layers_config, dropout):
        super().__init__()
        layers = []

        # 1. LAYER NORM ĐẦU VÀO (BẮT BUỘC)
        # Để cân bằng 'âm lượng' giữa ESM (hét to) và Ankh (nói nhỏ)
        layers.append(nn.LayerNorm(in_dim))

        prev_dim = in_dim
        for i, dim in enumerate(layers_config):
            layers.extend([
                nn.Linear(prev_dim, dim),
                nn.BatchNorm1d(dim), # Ổn định training
                nn.GELU(),           # Hiện đại hơn ReLU
                nn.Dropout(dropout)
            ])
            prev_dim = dim

        self.net = nn.Sequential(*layers)
        self.out_dim = prev_dim

    def forward(self, x):
        return self.net(x)

# Module lọc nhiễu (Attention)
class SEBlock(nn.Module):
    """Squeeze-and-Excitation Block để lọc nhiễu sau khi gộp"""
    def __init__(self, in_dim, reduction=4):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_dim, in_dim // reduction, bias=False),
            nn.ReLU(),
            nn.Linear(in_dim // reduction, in_dim, bias=False),
            nn.Sigmoid() # Tạo ra mask từ 0 đến 1
        )

    def forward(self, x):
        # x shape: [Batch, Dim]
        # Attention weight: [Batch, Dim]
        w = self.fc(x)
        # Nhân trọng số vào x: Cái nào quan trọng thì giữ, rác thì nhân với 0
        return x * w

class MultiModalNet(nn.Module):
    def __init__(self, input_dims_list, encoder_layers, dropout, num_classes):
        super().__init__()

        self.encoders = nn.ModuleList()
        self.fusion_input_dim = 0

        print("\n🏗️ Building Advanced Architecture:")

        # 1. Xây dựng các nhánh Encoder
        for i, in_dim in enumerate(input_dims_list):
            print(f"   ➤ Branch {i+1}: Input {in_dim} -> Output {encoder_layers[-1]}")
            enc = EncoderBlock(in_dim, encoder_layers, dropout)
            self.encoders.append(enc)
            self.fusion_input_dim += enc.out_dim

        print(f"   ➤ Fusion Dim: {self.fusion_input_dim}")

        # 2. SE-Block (Bộ lọc thông minh)
        self.attention_filter = SEBlock(self.fusion_input_dim)

        # 3. Layer tổng hợp cuối cùng
        self.head = nn.Sequential(
            nn.BatchNorm1d(self.fusion_input_dim),
            nn.Linear(self.fusion_input_dim, 512),
            nn.GELU(),
            nn.Dropout(dropout),

            # Thêm một lớp nữa để tăng khả năng học
            nn.Linear(512, 512),
            nn.GELU(),
            nn.Dropout(dropout),

            nn.Linear(512, num_classes)
        )

    def forward(self, inputs_list):
        features = []
        # Đi qua từng nhánh
        for i, encoder in enumerate(self.encoders):
            feat = encoder(inputs_list[i])
            features.append(feat)

        # Gộp lại (Concatenate)
        combined = torch.cat(features, dim=1)

        # LỌC NHIỄU (Điểm khác biệt lớn nhất)
        # Mạng sẽ tự học cách "tắt tiếng" các đặc trưng rác từ ProtT5 nếu nó thấy không cần thiết
        refined = self.attention_filter(combined)

        return self.head(refined)

train_paths = {k: v['train'] for k, v in CONFIG['EMBEDDINGS'].items()}
aspect_names = ['MFO', 'CCO', 'BPO']
temp_dataset = MultiSourceDataset(train_paths, indices=[0])
sample_inputs = temp_dataset[0][0] # Get inputs of first sample
INPUT_DIMS_LIST = [x.shape[0] for x in sample_inputs]
print(f"   ✓ Detected Input Dims: {INPUT_DIMS_LIST}")
del temp_dataset

# Lưu thông tin để dùng cho Inference
trained_models_meta = []

for aspect in aspect_names:
    terms = aspect_groups[aspect]
    if len(terms) == 0: continue

    print(f"\n" + "="*60)
    print(f"🧬 TRAINING ASPECT: {aspect} ({len(terms)} labels)")
    print("="*60)

    # --- A. PREPARE Y TENSOR CHO ASPECT NÀY ---
    print(f"   Shape mapping for {aspect}...")
    mlb = MultiLabelBinarizer(classes=terms, sparse_output=True)
    mlb.fit([terms])

    # Chỉ transform các nhãn thuộc Aspect này
    # Lưu ý: id_to_terms chứa tất cả, mlb sẽ tự ignore các nhãn không thuộc 'terms'
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        y_labels_list = [id_to_terms.get(pid, []) for pid in train_ids]
        y_sparse = mlb.transform(y_labels_list)

    # Convert to Tensor
    y_binary = y_sparse.astype(np.float32).toarray()
    y_tensor = torch.from_numpy(y_binary)

    # Label Smoothing
    if CONFIG['LABEL_SMOOTHING'] > 0:
        y_tensor.mul_(1 - CONFIG['LABEL_SMOOTHING']).add_(CONFIG['LABEL_SMOOTHING'] / len(terms))

    # Pos Weight (Optional - Re-calculate per aspect)
    weights_list = [ia_dict.get(t, 0.0) for t in terms]
    pos_weight = torch.tensor(weights_list, dtype=torch.float32).to(CONFIG['DEVICE'])

    # --- B. DATALOADERS ---
    indices = np.arange(len(y_tensor))
    train_idx, val_idx = train_test_split(indices, test_size=0.15, random_state=CONFIG['SEED'])

    train_dataset = MultiSourceDataset(train_paths, y_tensor, indices=train_idx)
    val_dataset   = MultiSourceDataset(train_paths, y_tensor, indices=val_idx)

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=True, num_workers=2)
    val_loader   = DataLoader(val_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=False, num_workers=2)

    # Auto detect input dims (Chỉ làm lần đầu hoặc làm lại cũng được)
    # sample_inputs, _ = train_dataset[0]
    # INPUT_DIMS_LIST = [x.shape[0] for x in sample_inputs]

    # --- C. BUILD MODEL ---
    # Reset model mới cho mỗi Aspect
    # model = MultiModalNet(
    #     input_dims_list = INPUT_DIMS_LIST,
    #     encoder_layers  = CONFIG['ENCODER_LAYERS'],
    #     dropout         = CONFIG['DROPOUT_RATE'],
    #     num_classes     = len(terms) # Output dim thay đổi theo Aspect
    # ).to(CONFIG['DEVICE'])

    # criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    # optimizer = optim.AdamW(model.parameters(), lr=CONFIG['LEARNING_RATE'], weight_decay=0.01)
    # scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

    # # --- D. TRAIN ---
    # best_val_loss = float('inf')
    model_save_path = f"{CONFIG['SAVE_DIR']}/model_{aspect}.pth"

    # for epoch in range(CONFIG['EPOCHS']):
    #     model.train()
    #     train_loss = 0
    #     for X_b, y_b in train_loader:
    #         X_b, y_b = [x.to(CONFIG['DEVICE']) for x in X_b], y_b.to(CONFIG['DEVICE'])
    #         optimizer.zero_grad()
    #         logits = model(X_b)
    #         loss = criterion(logits, y_b)
    #         loss.backward()
    #         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    #         optimizer.step()
    #         train_loss += loss.item()

    #     avg_train = train_loss / len(train_loader)

    #     model.eval()
    #     val_loss = 0
    #     with torch.no_grad():
    #         for X_b, y_b in val_loader:
    #             X_b, y_b = [x.to(CONFIG['DEVICE']) for x in X_b], y_b.to(CONFIG['DEVICE'])
    #             logits = model(X_b)
    #             loss = criterion(logits, y_b)
    #             val_loss += loss.item()

    #     avg_val = val_loss / len(val_loader)
    #     scheduler.step(avg_val)

    #     print(f"   Epoch {epoch+1:02d} | Train: {avg_train:.4f} | Val: {avg_val:.4f}")

    #     if avg_val < best_val_loss:
    #         best_val_loss = avg_val
    #         torch.save(model.state_dict(), model_save_path)

    # print(f"   🏆 Saved best {aspect} model to {model_save_path}")

    # Lưu meta data
    trained_models_meta.append({
        'aspect': aspect,
        'path': model_save_path,
        'terms': terms,
        'num_classes': len(terms)
    })

    # Dọn dẹp RAM
    # del model, train_loader, val_loader, train_dataset, val_dataset, y_tensor, y_sparse
    gc.collect()
    torch.cuda.empty_cache()

# ============================================================================
# 4. INFERENCE (MULTI-MODEL MERGE)
# ============================================================================
print("\n🔮 PREDICTING WITH MULTIPLE MODELS...")

QUOTAS = {
    'MFO': 5,  # Molecular Function thường ít nhãn hơn nhưng độ chính xác cao
    'BPO': 12,  # Biological Process rất nhiều nhãn, cần lấy nhiều hơn
    'CCO': 3   # Cellular Component ít nhãn
}

test_paths = {k: v['test'] for k, v in CONFIG['EMBEDDINGS'].items()}
test_ids = np.load(CONFIG['TEST_ID_PATH'])

# Dataset Test (Dùng chung cho cả 3 model)
test_dataset = MultiSourceDataset(test_paths)
test_loader = DataLoader(test_dataset, batch_size=CONFIG['BATCH_SIZE']*2, shuffle=False, num_workers=2)

# Load 3 Model vào list (Deep Learning model nhẹ, load cả 3 vào VRAM ok)
# Nếu VRAM quá yếu (T4 thường chịu được), thì có thể load-predict-unload từng cái
models_dict = {}

print("   Loading models...")
for meta in trained_models_meta:
    # Build lại kiến trúc
    aspect = meta['aspect']
    m = MultiModalNet(
        input_dims_list = INPUT_DIMS_LIST,
        encoder_layers  = CONFIG['ENCODER_LAYERS'],
        dropout         = CONFIG['DROPOUT_RATE'],
        num_classes     = meta['num_classes']
    ).to(CONFIG['DEVICE'])

    m.load_state_dict(torch.load(meta['path']))
    m.eval()
    models_dict[aspect] = {
        'model': m,
        'terms': meta['terms'] # Danh sách nhãn riêng của aspect đó
    }

print(f"   Loaded models for: {list(models_dict.keys())}")

submission_path = f"{CONFIG['SAVE_DIR']}/submission.tsv"
n_predictions = 0
current_idx = 0

with open(submission_path, 'w') as f:
    with torch.no_grad():
        for batch_idx, (X_b,) in enumerate(tqdm(test_loader, desc="Inferring")):
            X_b = [x.to(CONFIG['DEVICE']) for x in X_b]

            # Lấy ID của batch hiện tại
            start_idx = batch_idx * (CONFIG['BATCH_SIZE']*2)
            end_idx = start_idx + X_b[0].shape[0] # Dùng shape của tensor đầu tiên
            ids_batch = test_ids[start_idx : end_idx]

            # Dictionary lưu kết quả tạm: {pid: [(term, score), ...]}
            batch_results = {pid: [] for pid in ids_batch}

            # --- CHẠY TỪNG ASPECT RIÊNG BIỆT ---
            for aspect, item in models_dict.items():
                model = item['model']
                terms_list = item['terms']
                quota = QUOTAS.get(aspect, 15) # Lấy quota, mặc định 15

                # Predict
                logits = model(X_b)
                probs = torch.sigmoid(logits).cpu().numpy()

                # Lấy Top-K CHO TỪNG ASPECT
                for i, pid in enumerate(ids_batch):
                    p_row = probs[i]

                    # Lấy Top K của aspect này
                    if len(p_row) > quota:
                        ind = np.argpartition(p_row, -quota)[-quota:]
                    else:
                        ind = np.arange(len(p_row))

                    # Sort lại
                    ind = ind[np.argsort(p_row[ind])][::-1]

                    # Lưu vào buffer
                    for idx in ind:
                        score = p_row[idx]
                        if score > CONFIG['MIN_CONFIDENCE']:
                            term_name = terms_list[idx]
                            # Lưu tuple (term, score)
                            batch_results[pid].append((term_name, score))

            # --- GHI FILE ---
            for pid in ids_batch:
                # Lúc này batch_results[pid] chứa hỗn hợp MFO, BPO, CCO
                # Chúng ta KHÔNG sort lại theo điểm nữa (để tôn trọng quota)
                # Hoặc có thể sort nhẹ để file đẹp hơn, nhưng quan trọng là đã lấy đủ số lượng

                preds = batch_results[pid]
                # Optional: Sort tổng thể lần cuối để file submission đẹp
                preds.sort(key=lambda x: x[1], reverse=True)

                for term, score in preds:
                    f.write(f"{pid}\t{term}\t{score:.3f}\n")
                    n_predictions += 1

print(f"\n✅ DONE! Optimized Submission: {submission_path}")
print(f"✅ Total Predictions: {n_predictions:,}")

🚀 CAFA 6 - DUAL MODEL (ANKH + ESM) | Device: cuda

[1/5] Checking Files...
   ✓ Train IDs: 82404

[2/5] Processing Labels (IA Strategy)...
   ✓ Selected 10000 terms.
📖 Parsing OBO file: /content/cafa6_data/Train/go-basic.obo...
✅ Loaded aspect mapping for 48,101 terms.
📊 Aspect Distribution:
   - MFO (Molecular Function): 3,465 labels
   - CCO (Cellular Component): 1,612 labels
   - BPO (Biological Process): 4,923 labels

[5/5] Building Model...
   ✓ Detected Input Dims: [768, 1280, 1024]

🧬 TRAINING ASPECT: MFO (3465 labels)
   Shape mapping for MFO...

🧬 TRAINING ASPECT: CCO (1612 labels)
   Shape mapping for CCO...

🧬 TRAINING ASPECT: BPO (4923 labels)
   Shape mapping for BPO...

🔮 PREDICTING WITH MULTIPLE MODELS...
   Loading models...

🏗️ Building Advanced Architecture:
   ➤ Branch 1: Input 768 -> Output 512
   ➤ Branch 2: Input 1280 -> Output 512
   ➤ Branch 3: Input 1024 -> Output 512
   ➤ Fusion Dim: 1536

🏗️ Building Advanced Architecture:
   ➤ Branch 1: Input 768 -> Output 5

Inferring:   0%|          | 0/877 [00:00<?, ?it/s]


✅ DONE! Optimized Submission: /content/drive/MyDrive/CAFA6_Results/prott5_esm2_ankh_Run/submission.tsv
✅ Total Predictions: 3,872,236


In [22]:
# Chạy lệnh này trong một cell mới
!kaggle competitions submit \
    -c cafa-6-protein-function-prediction \
    -f /content/drive/MyDrive/CAFA6_Results/prott5_esm2_ankh_Run/submission.tsv \
    -m "esm2 ankh prot t5 chunk_aspect mf 5 bp 12 cc 3"

100% 88.9M/88.9M [00:01<00:00, 50.4MB/s]
Successfully submitted to CAFA 6 Protein Function Prediction

In [23]:
!kaggle competitions submissions -c cafa-6-protein-function-prediction

fileName        date                        description                                                            status                     publicScore  privateScore  
--------------  --------------------------  ---------------------------------------------------------------------  -------------------------  -----------  ------------  
submission.tsv  2025-12-16 04:41:43.003000  esm2 ankh prot t5 chunk_aspect mf 5 bp 12 cc 3                         SubmissionStatus.COMPLETE  0.268                      
submission.tsv  2025-12-16 04:35:42.487000  esm2 ankh prot t5 chunk_aspect mf 5 bp 12 cc 3                         SubmissionStatus.PENDING                              
submission.tsv  2025-12-15 04:33:37.193000  esm2 ankh prot t5 best_loss_model_path seed=17                         SubmissionStatus.COMPLETE  0.280                      
submission.tsv  2025-12-15 04:07:58.467000  esm2 ankh prot t5 best_loss_model_path top_k_labels=10k test_size=0.1  SubmissionStatus.COMPLETE  0.263   